# Docker Compose

## Docker Compose: la herramienta todo en uno

Todo lo que hemos hecho hasta ahora esta bien, pero es un poco tedioso. Para ello esta docker compose.

Si estas usando Windows esta instalada por defecto con docker desktop. Si estas usando Linux debes instalarla. Parece que ya la tengo. 

Docker Compose lo que nos permite es describir de forma declarativa la arquitectura de servicios que nuestra aplicacion necesita. Y se hace desde un pequeño archivo, para que docker corra casi todos los comandos que venimos ejecutando manualmente.

Vamos al proyecto en el **folder docker** donde hay un archivo llamado *docker-compose.yml*. Basicamente es una estructura declarativa.

![](https://i.imgur.com/rXZvpfz.png)

- services: indica los servicios que componen la aplicacion, en nuestro caso esta compuesta por dos servicios o microservicios: una que se llama **app** y otra que se llama **db** como en la seccion anterior.

- Donde se indica la imagen de mongo, le indico la version, pues ya tengo una imagen de dicha version.

Un servicio de compose es o viene siendo equivalente al concepto de container, donde **app** y **db** son dos contenedores. Aunque hay una sutil diferencia.

Porque un servicio, puede tener uno o mas contenedores de la misma imagen. Y tener por ejemplo mas contenedores de **app** para poder facilmente tener mas BW en nuestra aplicacion. 

- El statement *depends_on*, indicamos que dicho contenedor va a depender de el servicio *db*. Es decir, que **db** necesita ser creado antes que **app**

### Ejercicio.

Encontrar similitudes entre la figura anterior y los siguientes comandos usados anteriormente:

    docker run -d --name db mongo:5.0

    docker run -d --name app -p 3000:3000 --env MONGO_URL=mongodb://db:27017/test platziapp

### Aprendiendo a usar docker compose

#### Preparando el entorno

Empezemos por remover los contenedores previamente creados y evitar posibles conflictos:

![](https://i.imgur.com/39LyW8g.png)

    docker rm app db

Igualmente interesante, es lo descrito por el instructor, donde los procesos del ejercio anterior estan corriendo, y como al intentar crear un nuevo proceso usando los mismos puertos, pues va a generar un conflico

![](https://i.imgur.com/oMNxqh5.png)

#### Listo

Simplemente:

    docker compose up

Y empezara a correr y a llenar mi consola porque no lo corri en modo detach. Y empezara a mostrar los logs de ambas aplicaciones.

![](https://i.imgur.com/u9rCFJV.png)



    


- Crea los Containers, con un nombre propio, para evitar posibles conflictos con otros contenedores esten corriendo y tengan el mismo nombre. 

- Los containers empiezan por docker, no por docker, sino porque estoy en la carpeta docker 😏

- Fijate el orden crea los contenedores.

- A diferencia del instructor, no hay una evidencia haya creado una red

- y los atañe aparentemente a la misma red.

Si vamos al navegador en el *http://localhost:3000/* veremos esta corriendo 😎

Parece ser hay unas sutiles diferencias, de mis logs a los mostrados por el instructor. 

### Levantando el servicio en modo detach

    docker compose up -d

![](https://i.imgur.com/5QFMzye.png)

Y el resultado es el mismo:

![](https://i.imgur.com/ZkqFZLx.png)

#### Analizando 

![](https://i.imgur.com/HN9MRe8.png)

- app esta exponiendo el servicio en el puerto 3000 como era de esperarse y puedo accederlo desde mi maquina.

- pero db expone el puerto 27017 pero solo para la red interna.

¿Y cual es la red interna?


Inspeccionando cada una de las redes de docker encontre que:

    docker network inspect docker_default

![](https://i.imgur.com/Rz91spZ.png)

Los dos containers estan conectados en la red *docker default*



## Subcomandos de Docker Compose

👉 Importante aclarar que en el statement de la variable de entorno, en el compose file, **db** esta haciendo referencia al otro servicio, no necesito saber el nombre del contenedor(en perse) como fue fue el caso en la seccion de *docker network*

![](https://i.imgur.com/nS6sBTN.png)

### Ver los logs

Para ver lo de todos los servicios:

    docker compose logs 

Para ver solamente de un servicio:

    docker compose logs app

![](https://i.imgur.com/Y9nm16M.png)

Y para verlos en tiempo real, o a medida que se van generando:

    docker compose logs -f app

Y para el otro servicio obviamente:

    docker compose logs db

#### Corriendo un comando o proceso especifico en un contenedor

Muy facil, con exec, y en vez de colocar el nombre del contenedor, pues el nombre del servicio:

    docker compose exec app bash

Y efectivamente, vamos a poder ver nuestro codigo 

![](https://i.imgur.com/1WexGWK.png)






#### Borrando todo

    docker compose down 

E increiblemente, borra los contenedores tambien 😁

## Docker Compose como herramienta de desarrollo

